# 보스턴 주택 가격 데이터 세트를 Perceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
- 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [2]:
import pandas as pd
import numpy as np

In [1]:
from sklearn.datasets import load_boston

In [8]:
import warnings
warnings.filterwarnings(action='ignore')

In [9]:
boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['RPICE'] = boston.target
print(bostonDF.shape)
bostonDF.head(3)
# 사이킷런 1.0버전대에선 boston dataset이 deprecated됨
# 사이킷런 1.2 버전업이 되면 보스턴데이터셋이 없어진다함.

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,RPICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7


## Weight와 Bias의 Update 값을 계산하는 함수 생성
- w1은 RM(방의 개수) 피처의 weight값
- w2는 LSTAT(하위계층 비율) 피처의 weight값
- bias는 Bias
- N은 입력 데이터 건수

In [10]:
# gradient descent() 함수에서 반복적으로 호출되면서 update 될 weight/bias 값을 계산하는 함수
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨.
# 반환 값은 weight와 bias가 update되어야 할 값과 Mean Squeared Error 값을 loss로 반환
def get_update_weights_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):
    # 데이터 건수
    N = len(target)
    # 예측 값
    predicted = w1*rm + w2*lstat + bias
    # 실제값과 예측값의 차이
    diff = target - predicted
    # bias를 array 기반으로 구하기 위해서 설정
    bias_factors = np.ones((N, ))
    
    # weight와 bias를 얼마나 update할 것인지를 계산
    w1_update = -(2/N)*learning_rate*(np.dot(rm.T, diff))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat.T, diff))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff))
    
    # Mean Squared Error값을 계산
    mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환
    return bias_update, w1_update, w2_update, mse_loss

## Gradient Descent를 적용하는 함수 생성
- iter_epochs 수만큼 반복적으로 get_update_weights_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update 적용